In [113]:
import re
content = open('cgn_canada_csv_eng.csv','r').read()
places = set(re.findall('^[^,]*,([^,]*),', content, re.MULTILINE))
places.add('Canada')
print(f'Number of Canada places: {len(places):,}')
places = set([place.lower() for place in places if len(place)>3])
print(f'Number of Canada places: {len(places):,}')

Number of Canada places: 239,685
Number of Canada places: 239,487


In [114]:
import os

GM1 = '/home/ec2-user/SageMaker/data/GM_all_1945_1956/'
GM2 = '/home/ec2-user/SageMaker/data/GM_all_1957-1967/'

files = [GM1+f for f in os.listdir(GM1)]+[GM2+f for f in os.listdir(GM2)]
print(f'len(files)={len(files):,}')

len(files)=2,057,868


In [115]:
import re
import sys
sys.path.append('..')
from utils.tdmstudio import TDMStudio
import spacy
nlp = spacy.load('en_core_web_sm', disable=['textcat', 'lemmatizer', 'ner', 'parser', 'tagger'])

def in_canada(text):
    text = text.lower()
    for token in nlp.tokenizer(text):
        if token.text in places:
            return True
    return False

from threading import Lock
wlock = Lock()

def process_file(file_):
    title, text= TDMStudio.get_title_and_text(file_)
    if not title is None and not text is None:
        id_ = file_.split('/')[-1][:-4]
        in_canada_ = in_canada(f'{title}. {text}')
        wlock.acquire()
        writer1.write(f"{id_};{in_canada_}\n")
        writer1.flush()
        wlock.release()


In [122]:
%%time
from utils.general import info, ok
import concurrent.futures
import datetime
writer1 = open('in_canada.csv', 'w')
writer2 = open('done_in_canada.txt', 'w')
writer2.write(f'{datetime.datetime.now()} Starting...\n')

info('Starting...')

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(process_file, files, chunksize=100000)
    
    
writer2.write(f'{datetime.datetime.now()} Done!')
writer1.close()
writer2.close()
ok('Done!')

2022-03-23 15:21:32.067056 [ INFO  ] Starting...
2022-03-23 16:37:30.658158 [  OK   ] Done!
CPU times: user 1h 21min 18s, sys: 11min 5s, total: 1h 32min 23s
Wall time: 1h 15min 58s


In [120]:
!grep True in_canada.csv  |wc -l

9450


In [121]:
!grep False in_canada.csv  |wc -l

503


In [40]:
'Bedford' in places

False

In [31]:
places


['canada',
 'abbotsford',
 'acton vale',
 'airdrie',
 'alma',
 'amos',
 'amqui',
 'armstrong',
 'baie-comeau',
 "baie-d'urfé",
 'baie-saint-paul',
 'barkmere',
 'barrie',
 'bathurst',
 'beaconsfield',
 'beauceville',
 'beauharnois',
 'beaumont',
 'beaupré',
 'bedford',
 'belleterre',
 'belleville',
 'beloeil',
 'berthierville',
 'blainville',
 'bois-des-filion',
 'boisbriand',
 'bonaventure',
 'boucherville',
 'brampton',
 'brandon',
 'brantford',
 'brockville',
 'bromont',
 'brooks',
 'brossard',
 'brownsburg-chatham',
 'burlington',
 'burnaby',
 'bécancour',
 'calgary',
 'cambridge',
 'campbell river',
 'campbellton',
 'camrose',
 'candiac',
 'cap-chat',
 'cap-santé',
 'carignan',
 'carleton-sur-mer',
 'castlegar',
 'causapscal',
 'chambly',
 'chandler',
 'chapais',
 'charlemagne',
 'charlottetown',
 'chatham-kent',
 'chibougamau',
 'chilliwack',
 'château-richer',
 'châteauguay',
 'city of west kelowna',
 'clarence-rockland',
 'clermont',
 'coaticook',
 'cold lake',
 'colwood',
 'co